In [1]:
from xgboost import XGBClassifier
import pandas as pd
import numpy as np

In [2]:
train = pd.read_csv("./train3.csv")
train.head()

,content,label,tokens
0,什么是电气一次设备和一次回路,1.0,"['什么', '是', '电气', '一次', '设备', '和', '一次', '回路']"
1,一次设备是指直接生产输送和分配电能的高压,0.0,"['一次', '设备', '是', '指', '直接', '生产', '输送', '和', ..."
2,什么是电气二次设备和二次回路,1.0,"['什么', '是', '电气', '二次', '设备', '和', '二次', '回路']"
3,二次设备是指对一次设备的工作进行监测控制,0.0,"['二次', '设备', '是', '指', '对', '一次', '设备', '的', '..."
4,哪些回路属于连接保护装置的二次回路,1.0,"['哪些', '回路', '属于', '连接', '保护装置', '的', '二次', '回路']"


In [3]:
def analyzer(x):
    return x.split()
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(smooth_idf=True,
                                analyzer=analyzer,
                                ngram_range=(1, 1),
                                min_df=0.1, norm='l1')
sparse_vector = vectorizer.fit_transform(train.tokens.apply(lambda x: ' '.join(x)).tolist())
label = train.label.to_list()

In [19]:
model = XGBClassifier(booster="gbtree",objective="binary:logistic")
from sklearn.model_selection import GridSearchCV,GroupKFold
gsCv = GridSearchCV(model,
                   {'max_depth': [1,2,3],
                    'n_estimators':[120],
                   'min_child_weight':[1],
                   'gamma':[0.01],
                   'subsample':[0.5,0.7,1],
                   })
gsCv.fit(sparse_vector,label)
gsCv.best_params_,gsCv.best_score_

In [32]:
new_model = XGBClassifier(booster="gbtree",objective="binary:logistic",**gsCv.best_params_)
gsCv2 = GridSearchCV(new_model, 
                   {'learning_rate ': [0.1,1],
                   'n_estimators':[100,110,120,130]})
gsCv2.fit(sparse_vector,label)
gsCv2.best_params_,gsCv2.best_score_

({'learning_rate ': 1, 'n_estimators': 100}, 0.9873015873015873)

In [36]:
res_model = XGBClassifier(booster="gbtree",objective="binary:logistic",gamma=0.01,verbosity=1,max_depth=2,min_child_weight=1,n_estimators=100,learning_rate=1,subsample=0.5)
res_model = res_model.fit(sparse_vector,label,early_stopping_rounds=10, eval_metric="error",eval_set=[(sparse_vector, label)])

[0]	validation_0-error:0.04924
Will train until validation_0-error hasn't improved in 10 rounds.
[1]	validation_0-error:0.02859
[2]	validation_0-error:0.02621
[3]	validation_0-error:0.01509
[4]	validation_0-error:0.01509
[5]	validation_0-error:0.01430
[6]	validation_0-error:0.00874
[7]	validation_0-error:0.01271
[8]	validation_0-error:0.00874
[9]	validation_0-error:0.00635
[10]	validation_0-error:0.00794
[11]	validation_0-error:0.00556
[12]	validation_0-error:0.00477
[13]	validation_0-error:0.00477
[14]	validation_0-error:0.00477
[15]	validation_0-error:0.00397
[16]	validation_0-error:0.00397
[17]	validation_0-error:0.00477
[18]	validation_0-error:0.00318
[19]	validation_0-error:0.00238
[20]	validation_0-error:0.00079
[21]	validation_0-error:0.00159
[22]	validation_0-error:0.00159
[23]	validation_0-error:0.00159
[24]	validation_0-error:0.00159
[25]	validation_0-error:0.00159
[26]	validation_0-error:0.00079
[27]	validation_0-error:0.00079
[28]	validation_0-error:0.00079
[29]	validation_

In [38]:
import joblib
model_path = "./model1.model"
joblib.dump(res_model, model_path)

['./model1.model']